This notebook cleans .txt data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob,string
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_config_file(world_readable=True)

def cleanSoarExam (data, examNum, fileType='flat',
                   colSpec = [(0,9),(10,21),(22,28),(29,30),(30,32),(32,34),(34,36),(36,38),(39,41),(41,68)],
                   soarSessions=[]):

    if fileType != 'flat':
        print("\n{} fileType is not currently supported.".format(fileType))
    else:
        try:
            df=pd.read_fwf(data,colSpec,names=['tuid','last','first','middle','unnamed1','unnamed2','unnamed3','soar','ncorrect','item'])
        except:
            "print(\nCould not find file {} or {} was not acceptable value for colSpec)".format(data,colSpec)
        #try:
        #    df.columns = ['tuid','last','first','middle','unnamed1','unnamed2','unnamed3','soar','ncorrect','item']
        #except:
        #    print("\nColumn number != 10.\n{}".format(len(df.columns)))
        try:
            df['examNumber']=examNum
            numbers=pd.Series(list(range(28))).astype(str)
            itemNames= 'item_'+ numbers[1:]
            itemData=df.item.apply(lambda i: pd.Series(list(i)))
            itemData.columns=itemNames
            df=df.merge(itemData,'outer',left_index=True,right_index=True).drop('item',axis=1)
        except:
            print("\nUnhandled exception encountered.")
        if soarSessions ==np.nan:
            df.loc['soarType']='other'
        else:
            df.loc[df.soar.isin(soarSessions),'soarType']='mine'
            df.loc[~df.soar.isin(soarSessions),'soarType']='other'
            df.loc[df.tuid=='NNNNNNNNN','soarType']='key'
    return df
        

Here we need to write a function to determine how many students get each item wrong.

In [2]:
# import pandas as pd
# import numpy as np

def nwrong (x,key):
    """
    :param x: pd.Series
    :param key: ~None
    Takes a pandas series and a specified value and returns the number of values 
    in the series which do not match the specified value."""
    assert isinstance(x, pd.Series)
    x = x.astype(str)
    key = str(key)
    return x[x!=key].count()
    

In [3]:
list(string.ascii_lowercase)[0]

'a'

# Setting up notebook

In [4]:
import pandas as pd
import numpy as np
import os,glob

pd.options.display.max_columns=50

# Getting Exam Data

Here we define paths for getting exam data and outputting results

In [5]:
homesource = 'S:/Chris/Temple/Biol1111/Fall 2018/Raw Data/Exam 2'
homeoutputFolder = 'S:/Chris/Temple/Biol1111/Fall 2018/Results/Exam 2'
worksource = 'C:/Users/tuh27554/Documents/BIOL1111/Fall 2018/Raw Data/Exam 2'
workoutputFolder = 'C:/Users/tuh27554/Documents/BIOL1111/Fall 2018/Results/Exam 2'

Here we get a list of source data files. For this notebook we will use the \*.txt files.

In [6]:
os.chdir(homesource)
files = glob.glob('*.txt')
print(files)

['E2V21Raw.txt', 'E2V22Raw.txt']


 And now we read in and clean those data using the *cleanSoarExam* function.

In [7]:
df1 = cleanSoarExam(files[0],examNum=1,soarSessions=[81])
print('Version 1 of the exam has {} students.'.format(df1.shape[0]-1))
df2 = cleanSoarExam(files[1],examNum=1,soarSessions=[81])
print('Version 2 of the exam has {} students.'.format(df2.shape[0]-1))
os.chdir(homeoutputFolder)

Version 1 of the exam has 121 students.
Version 2 of the exam has 118 students.


# Analyzing Exam Data

## Determining the most problematic items for the class

We need to identify the item columns over which to apply *nwrong*.

In [8]:
itemcols1 = df1.columns[df1.columns.str.contains('item_')]
itemcols2 = df2.columns[df2.columns.str.contains('item_')]

In [18]:
df.head()

,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType
1,915613829,GILL,SAMNIT,NaN,21.0,10.0,49.0,73.0,21,1,2,4,1,4,3,3,1,4,4,3,4,2,1,1,2,3,4,2,3,3,2,2,1,3,2,2,3,other
2,915063949,LU,WENROU,NaN,21.0,10.0,31.0,71.0,14,1,2,3,1,4,3,4,2,3,2,4,3,2,1,1,1,1,4,2,4,3,1,1,2,3,2,2,1,other
3,915397247,UNG,NANCY,NaN,21.0,11.0,9.0,71.0,11,1,1,4,1,4,3,2,3,1,2,2,3,2,2,3,2,3,4,4,3,4,4,4,3,2,4,3,1,other
4,915629319,ALQALLAF,ALI,NaN,21.0,10.0,50.0,73.0,25,1,2,4,1,4,3,3,1,4,2,1,3,2,1,1,2,3,4,4,3,1,2,1,1,1,2,2,3,other
5,915468481,NICHOLSON,JO,M,21.0,10.0,40.0,83.0,21,1,1,4,1,4,3,3,1,4,4,1,3,2,1,1,2,1,4,2,1,4,2,1,1,3,2,2,3,other


In [19]:
df1.loc[(df1.soarType=='key') & (df1.item_1!=df1.loc[df1.soarType=='key','df1.item_1'])].ncorrect +1

KeyError: 'the label [df1.item_1] is not in the [columns]'

Now we apply the function to determine the number of students who answered incorrectly for version 1 and version 2 separately. We will only print one of these here for an example.

In [9]:
v1wrong = df1.loc[df1.soarType.isin(['mine','key']),itemcols1].apply(lambda x: nwrong(x=x[1:],key=x[0]))
v1wrong = v1wrong.rename({'item_1':'item_1a'})
v1wrong

item_1a    7
item_2     5
item_3     2
item_4     0
item_5     5
item_6     0
item_7     4
item_8     4
item_9     3
item_10    9
item_11    3
item_12    4
item_13    3
item_14    2
item_15    4
item_16    5
item_17    1
item_18    6
item_19    4
item_20    7
item_21    4
item_22    8
item_23    2
item_24    9
item_25    2
item_26    2
item_27    8
dtype: int64

In [10]:
v2wrong = df2.loc[df2.soarType.isin(['mine','key']),itemcols2].apply(lambda x: nwrong(x=x[1:],key=x[0]))
v2wrong = v2wrong.rename({'item_1':'item_1b'})
v2wrong

item_1b     7
item_2      2
item_3      2
item_4      1
item_5      5
item_6      3
item_7      9
item_8      8
item_9      8
item_10    12
item_11     6
item_12    11
item_13    10
item_14     8
item_15     4
item_16     8
item_17     0
item_18     7
item_19     4
item_20    13
item_21    11
item_22    13
item_23     4
item_24    13
item_25     6
item_26     2
item_27     5
dtype: int64

Adding these lists together we get the total number wrong on each item.  If we sort the resulting series in descending order, we will have the guide we need to determine the order for discussing the items in class.

In [11]:
totalwrong = v1wrong + v2wrong
orderedwrong = totalwrong.sort_values(ascending=False)
# orderedwrong.loc[~orderedwrong.isna()] = orderedwrong.loc[~orderedwrong.isna()].astype(int)
orderedwrong

item_24    22.0
item_10    21.0
item_22    21.0
item_20    20.0
item_12    15.0
item_21    15.0
item_18    13.0
item_7     13.0
item_13    13.0
item_16    13.0
item_27    13.0
item_8     12.0
item_9     11.0
item_14    10.0
item_5     10.0
item_11     9.0
item_15     8.0
item_19     8.0
item_25     8.0
item_2      7.0
item_23     6.0
item_26     4.0
item_3      4.0
item_6      3.0
item_17     1.0
item_4      1.0
item_1a     NaN
item_1b     NaN
dtype: float64

Let's also print the letters for the correct answers for each item.

In [12]:
v1keys = df1.loc[df1.soarType=='key',itemcols1].apply(lambda x: list(string.ascii_uppercase)[int(x)-1])
v1keys.to_csv('e2v1keys.csv')
v1keys
# v2wrong = df2.loc[df2.soarType.isin(['mine','key']),itemcols2].apply(lambda x: nwrong(x=x[1:],key=x[0]))

item_1     B
item_2     D
item_3     A
item_4     D
item_5     C
item_6     C
item_7     A
item_8     D
item_9     B
item_10    A
item_11    C
item_12    B
item_13    A
item_14    A
item_15    B
item_16    C
item_17    D
item_18    B
item_19    C
item_20    A
item_21    B
item_22    A
item_23    A
item_24    D
item_25    B
item_26    B
item_27    C
dtype: object

Here we need to write a function to determine how many students get each item wrong.

In [13]:
v2keys = df1.loc[df1.soarType=='key',itemcols2].apply(lambda x: list(string.ascii_uppercase)[int(x)-1])
v2keys.to_csv('e2v2keys.csv')
v2keys

item_1     B
item_2     D
item_3     A
item_4     D
item_5     C
item_6     C
item_7     A
item_8     D
item_9     B
item_10    A
item_11    C
item_12    B
item_13    A
item_14    A
item_15    B
item_16    C
item_17    D
item_18    B
item_19    C
item_20    A
item_21    B
item_22    A
item_23    A
item_24    D
item_25    B
item_26    B
item_27    C
dtype: object

## Evaluating students performace 

Now we look at scores by section.  For this we can rename *unnamed1* to *version* to keep track of the versions and append the 2 dfs. 

In [14]:
df = df1.append(df2)
print(df.shape)
df.to_csv('merged exam 2 results.csv')
# df.head()

(241, 38)


We can drop the "key" data and store as a separate variable,*maxscore*, the maximum possible value for ncorrect.

In [15]:
maxscore = df.ncorrect.max()
dfkey = df.loc[df.soarType=='key',:]
df = df.loc[df.soarType!='key',:]
print('maxscore:{}'.format(maxscore))

maxscore:27


In [16]:
df.loc[df.item_1!=dfkey.item_1,'ncorrect'] = df.loc[df.item_1!=dfkey.item_1,'ncorrect'] + 1

ValueError: Can only compare identically-labeled Series objects

We will use this *df* for the rest of our analysis.

In [ ]:
def iqr(x):
    return x.quantile(.75) - x.quantile(.25)

def percentile25(x):
    return x.quantile(.25)

def percentile75(x):
    return x.quantile(.75)


In [ ]:
df.head()

In [ ]:
dfkey.soarType.unique()

In [ ]:
scores = df.groupby(['soar']).ncorrect\
.agg(['count','min',percentile25,percentile75,'max','median',iqr,'mean','std'])\
.sort_values('median', ascending =False)
scores.to_csv('Exam2Score breakdown.csv')
scores

Now let's look at the distribution of individuals who scored above the 75-percentile in their class.  This gives us an idea of how distributed the high scores are in each section.

In [ ]:
highAchieverslocal = df.groupby('soar').ncorrect.apply(lambda x: x.loc[x>x.quantile(.75)].count()).reset_index()\
.rename(columns = {'ncorrect':'nAbove75p'})
highAchieverslocal.soar = highAchieverslocal.soar.astype(int)
highAchieverslocal.soar = highAchieverslocal.soar.astype(str)
highAchieverslocal

In [ ]:

data = [go.Bar(x = highAchieverslocal.soar, y = highAchieverslocal.nAbove75p)]
layout = go.Layout(
    title = 'Higher Acheivers Local',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'SOAR Section',
        titlefont = dict(
            size = 18),
        tickangle = 90),
    yaxis = dict(
        title = 'Number of High Achievers',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Bar of High Achievers 2018E21111')

Now let's look at the distribution of individuals who scored above the 75-percentile across the entire class.  This gives us an idea of how distributed the high scores are across sections.

In [ ]:
highAchieversGlobal = df.loc[df.ncorrect>df.ncorrect.quantile(.75)]\
.groupby('soar').ncorrect.count().reset_index()\
.rename(columns = {'ncorrect':'nAbove75p'})
print('The overall all median and 75 percentile were {} and {}, \
respectively.'.format(df.ncorrect.median(),df.ncorrect.quantile(.75)))
highAchieversGlobal

## Vizualization

In [ ]:
data = [go.Bar(x = highAchieversGlobal.soar,y = highAchieversGlobal.nAbove75p)]
layout = go.Layout(
    title = 'Higher Acheivers Global',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'SOAR Section',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of High Achievers',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Bar of High Achievers Global 2018E21111')